# Accuracy Test

This test measures the accuracy of the decision engine with images from the validation set of the image classifier.
The ontology is fed with sensor data that is in the range specified in the expert rules for the ground truth label with a probability $\geq 60\%$.

In [7]:
import datetime
from pathlib import Path
import time

import pandas
import pandas as pd
import requests
from tqdm.notebook import tqdm
from numpy.random import default_rng
import numpy as np

Define the sensor value ranges, so they can be used as boundaries of a uniform distribution.
The boundaries are chosen in such a way that the expert rule will be fulfilled with a probability p where $0.6 \leq p \leq 1$.
This should simulate, that a disease is likely to be encountered in a favorable environment for the disease.

In [8]:
DISEASES = [
    'cassava_bacterial_blight',
    'cassava_brown_streak_disease',
    'cassava_green_mottle',
    'cassava_mosaic_disease',
    'healthy',
]

SENSOR_RANGES = {
    'cassava_bacterial_blight': {
        'SoilMoisture': (0.1, 1),
        'SoilPH': (6.3, 7.5),
        'SoilTemperature': (23, 31),
    },
    'cassava_brown_streak_disease': {
        'RelativeHumidity': (0.65, 0.88),
        'SoilMoisture': (0.0, 1.0),
        'SoilTemperature': (5, 33),
    },
    'cassava_green_mottle': {
        'SoilMoisture': (0.55, 1.0),
        'RelativeHumidity': (0.6, 1.0),
        'SoilTemperature': (24, 40)
    },
    'cassava_mosaic_disease': {
        'Temperature': (20, 50),
        'SoilMoisture': (0.1, 1),
        'SoilTemperature': (18, 34),
        'RelativeHumidity': (0.7, 1)
    },
    'healthy': {
        'Temperature': (0, 50),
        'SoilMoisture': (0.2, 0.8),
        'SoilTemperature': (0, 40),
        'RelativeHumidity': (0.2, 0.8),
        'SoilPH': (3.0, 10.0),
    }
}

SENSOR_RULE_RANGES = {
    'cassava_bacterial_blight': {
        'SoilMoisture': (0.3, 1),
        'SoilPH': (6.5, 7.2),
        'SoilTemperature': (25, 30),
    },
    'cassava_brown_streak_disease': {
        'RelativeHumidity': (0.7, 0.85),
        'SoilMoisture': (0.1, 1.0),
        'SoilTemperature': (10, 32),
    },
    'cassava_green_mottle': {
        'SoilMoisture': (0.7, 1.0),
        'RelativeHumidity': (0.7, 1.0),
        'SoilTemperature': (27, 40)
    },
    'cassava_mosaic_disease': {
        'Temperature': (30, 50),
        'SoilMoisture': (0.3, 1),
        'SoilTemperature': (20, 32),
        'RelativeHumidity': (0.8, 1)
    },
    'healthy': {
        'Temperature': (0, 50),
        'SoilMoisture': (0.2, 0.8),
        'SoilTemperature': (0, 40),
        'RelativeHumidity': (0.2, 0.8),
        'SoilPH': (3.0, 10.0),
    }
}

PLANTS_FOR_DISEASES = {
    'cassava_bacterial_blight': 1,
    'cassava_brown_streak_disease': 2,
    'cassava_green_mottle': 3,
    'cassava_mosaic_disease': 4,
    'healthy': 5
}

SOIL_PROPERTIES = [
    'SoilMoisture',
    'SoilTemperature',
    'SoilPH'
]

FIELD_PROPERTIES = [
    'Temperature',
    'RelativeHumidity'
]

Configure service endpoints.

In [9]:
ONTO_URL = 'http://localhost:8000'
DECISION_URL = 'http://localhost:9000'
DATA_PATH = 'data/train/'
DISTRIBUTION = "uniform"
FULL_TEST = True

rng = default_rng(12345)
VALIDATION_DATASET = "./checkpoints/fold_0.csv"
valid_df = pd.read_csv(VALIDATION_DATASET)
valid_df['image_id'] = list(map(Path(DATA_PATH).joinpath, valid_df['image_id']))

      Unnamed: 0                       image_id  label  source  fold
0              5      data/train/1000837476.jpg      3    2020     0
1              8      data/train/1001723730.jpg      4    2020     0
2             11       data/train/100204014.jpg      3    2020     0
3             23      data/train/1004672608.jpg      3    2020     0
4             28      data/train/1005200906.jpg      2    2020     0
...          ...                            ...    ...     ...   ...
5263       26295   data/train/train-cbb-164.jpg      0    2019     0
5264       26298   data/train/train-cmd-324.jpg      3    2019     0
5265       26301  data/train/train-cmd-2096.jpg      3    2019     0
5266       26303   data/train/train-cmd-335.jpg      3    2019     0
5267       26316  data/train/train-cmd-2258.jpg      3    2019     0

[5268 rows x 5 columns]


In [10]:
def get_range(_property: str, disease: str):
    try:
        return SENSOR_RANGES[disease][_property]
    except KeyError:
        return SENSOR_RANGES['healthy'][_property]

def generate_observation(_property: str, disease: str, distribution='uniform'):
    _range = get_range(_property, disease)
    if distribution == "uniform":
        value = rng.uniform(*_range)
    elif distribution == "normal":
        value = rng.normal(np.mean(_range), (_range[1] - _range[0]/8))
    return {
       'timestamp': str(datetime.datetime.now()),
       'value': value,
       'observed_property': _property
    }

def simulate_sensors(disease: str):
    id = PLANTS_FOR_DISEASES[disease]
    field_url = f"{ONTO_URL}/fields/{id}/observations"
    soil_url = f"{ONTO_URL}/soils/{id}/observations"

    headers = {'content-type': 'application/json',
               'accept': 'application/json'}

    for _property in SOIL_PROPERTIES:
        observation = generate_observation(_property, disease, DISTRIBUTION)
        requests.post(soil_url, json=observation, headers=headers)

    for _property in FIELD_PROPERTIES:
        observation = generate_observation(_property, disease, DISTRIBUTION)

        requests.post(field_url, json=observation, headers=headers)

In [11]:
def test_decision_engine() -> pandas.DataFrame:
    results = []

    headers = {
               'accept': 'application/json'}

    total_samples = len(valid_df) if FULL_TEST else 100

    for i, sample in tqdm(valid_df.iterrows(), total=total_samples):
        disease = DISEASES[sample['label']]
        image = sample['image_id']
        plant_id = PLANTS_FOR_DISEASES[disease]

        # test fewer samples for tuning
        if i >= 100 and not FULL_TEST:
            break

        simulate_sensors(disease)

        start_time = time.time()
        with open(image, 'rb') as f:
            files = {'image': (image.name, f, 'image/jpg')}
            response = requests.post(DECISION_URL + f"/plants/{plant_id}/predict-disease",
                                     files=files, headers=headers)
            end_time = time.time()
            decision_time = end_time - start_time
            response.raise_for_status()

            pred = response.json()

            results.append({
                'ground_truth': disease,
                'prediction': pred['disease'],
                'visual_certainty': pred['visual_certainty'],
                'knowledge_certainty': pred['knowledge_certainty'],
                'image_classification_time': pred['image_classification_time'],
                'reasoner_time': pred['reasoner_time'],
                'decision_time': decision_time,
                'image': str(image)
            })

    return pd.DataFrame.from_records(results)

In [12]:
result = test_decision_engine()

  0%|          | 0/5268 [00:00<?, ?it/s]

In [13]:
matches = result[result['ground_truth'] == result['prediction']]
accuracy = len(matches)/len(result)
print(f"Accuracy is {accuracy}")
errors = result[result['ground_truth'] != result['prediction']]
print(errors)


Accuracy is 0.9050873196659074
                      ground_truth                    prediction  \
1                          healthy      cassava_bacterial_blight   
4             cassava_green_mottle                       healthy   
14                         healthy        cassava_mosaic_disease   
28          cassava_mosaic_disease                       healthy   
58            cassava_green_mottle                       healthy   
...                            ...                           ...   
5202        cassava_mosaic_disease  cassava_brown_streak_disease   
5204      cassava_bacterial_blight  cassava_brown_streak_disease   
5216          cassava_green_mottle  cassava_brown_streak_disease   
5227  cassava_brown_streak_disease        cassava_mosaic_disease   
5234  cassava_brown_streak_disease                       healthy   

      visual_certainty  knowledge_certainty  image_classification_time  \
1             0.997882             0.250000                   0.948990   
4   

uniform distribution + volo_d2_384

| Image Weight | Knowledge Weight | Accuracy 100 | Accuracy Full | classifier |
|--------------|------------------|--------------|---------------|------------|
| 0.5          | 0.5              | 0.9          | 0.905         | 0.8964     |
| 0.4          | 0.6              | 0.88         | -             | -          |
| 0.3          | 0.7              | 0.75         | -             | -          |
| 0.6          | 0.4              | 0.89         | -             | -          |
| 0.7          | 0.3              | 0.89         | -             | -          |

In [14]:
result.to_csv("./times.csv", columns=['decision_time',
                                      'image_classification_time',
                                      'reasoner_time'] )

print(f"avg_decision_time: {result['decision_time'].mean()}")
print(f"avg_classification_time: {result['image_classification_time'].mean()}")
print(f"avg_reasoner_time: {result['reasoner_time'].mean()}")

avg_decision_time: 3.851396314782814
avg_classification_time: 0.9235793290123762
avg_reasoner_time: 2.916423389426487


```
avg_decision_time: 3.6626797550796444
avg_classification_time: 0.21957266697071337
avg_reasoner_time: 3.4327719566600225
```